In [ ]:
import os
# Google Cloud Speech-to-Text 인증 키 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Your_Api_Key.json"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
BUCKET_NAME = "ccg70098"
FFMPEG_PATH = r"C:\Users\ccg70\OneDrive\desktop\넥슨 프로젝트\ffmpeg-7.1-essentials_build\bin\ffmpeg.exe"

: 

# 유튜브 영상 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def crawl_inven_post_with_selenium(url):
    options = Options()
    options.add_argument("--headless")  # 브라우저 UI를 보지 않으려면 headless 모드 사용
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)

    # 페이지 로딩 대기
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.comment"))
    )

    # 댓글 크롤링
    comments = []
    comment_elements = driver.find_elements(By.CSS_SELECTOR, "div.comment span.content.cmtContentOne")
    for comment_element in comment_elements:
        comments.append(comment_element.text.strip())

    driver.quit()
    return comments

# URL 리스트 설정 (추가할 URL 목록)
urls = [
    "https://www.inven.co.kr/board/maple/2304/41944",
    "https://www.inven.co.kr/board/maple/2304/41910?p=2",
    "https://www.inven.co.kr/board/maple/2304/41927?my=chu",
    "https://www.inven.co.kr/board/maple/2304/41807?my=chu",
]

for url in urls:
    comments = crawl_inven_post_with_selenium(url)
    print(f"Comments for {url}: {comments}")

NameError: name 'API_KEY' is not defined

# 인벤 게시글 크롤링

In [29]:
import requests
from bs4 import BeautifulSoup
import json
import os

def crawl_inven_post(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    try:
        title = soup.find('div', class_='articleTitle').text.strip()
    except AttributeError:
        title = "제목을 찾을 수 없습니다"
        print("제목 요소를 찾을 수 없습니다.")
    
    try:
        date = soup.find('div', class_='articleDate').text.strip()
    except AttributeError:
        date = "날짜를 찾을 수 없습니다"
        print("날짜 요소를 찾을 수 없습니다.")
    
    try:
        content = soup.find('div', id='powerbbsContent').text.strip()
    except AttributeError:
        content = "내용을 찾을 수 없습니다"
        print("내용 요소를 찾을 수 없습니다.")
    
    post_data = {
        'title': title,
        'date': date,
        'content': content
    }
    
    return post_data

# URL 리스트 설정 (추가할 URL 목록)
urls = [
    "https://www.inven.co.kr/board/maple/2304/41944",
    "https://www.inven.co.kr/board/maple/2304/41910?p=2",
    "https://www.inven.co.kr/board/maple/2304/41927?my=chu",
    "https://www.inven.co.kr/board/maple/2304/41807?my=chu",
    
]

# JSON 파일 경로
json_file_path = 'inven_posts.json'

# 기존 JSON 파일에서 데이터를 불러오거나 새 리스트 생성
if os.path.exists(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
else:
    data = []

# 각 URL을 크롤링하여 데이터를 리스트에 추가
for url in urls:
    post_data = crawl_inven_post(url)
    data.append(post_data)

# JSON 파일로 저장 (기존 데이터에 추가된 데이터를 함께 저장)
with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("크롤링 완료 및 JSON 파일에 데이터 추가 저장됨")


크롤링 완료 및 JSON 파일에 데이터 추가 저장됨


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import logging
from concurrent.futures import ThreadPoolExecutor  # 병렬 처리를 위한 모듈 추가

# 로깅 수준 설정
logging.getLogger('selenium').setLevel(logging.WARNING)

# 댓글 크롤링 함수 정의 (Selenium 사용)
def crawl_comments_with_selenium(url):
    options = Options()
    options.add_argument("--disable-gpu")  # GPU 가속 비활성화
    options.add_argument("--no-sandbox")  # 샌드박스 비활성화
    options.add_argument("--headless")  # 브라우저 UI를 보지 않으려면 headless 모드 사용
    options.add_argument("--log-level=3")  # ERROR 레벨 이상의 로그만 출력
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)

    comments = []
    try:
        # 페이지 로딩 대기
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.comment"))
        )

        # 댓글 크롤링
        comment_elements = driver.find_elements(By.CSS_SELECTOR, "div.comment span.content.cmtContentOne")
        for comment_element in comment_elements:
            comments.append(comment_element.text.strip())

    except Exception:
        # 오류 메시지 출력하지 않음
        comments = []  # 댓글이 없거나 오류가 발생한 경우 빈 리스트 반환

    finally:
        driver.quit()

    return comments


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
}

# 게시글 크롤링 함수 정의 (BeautifulSoup 사용)
def crawl_post(post_id):
    url = f"https://www.inven.co.kr/board/maple/2300/{post_id}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, "html.parser")

        # 제목 추출
        title = soup.find('div', class_='articleTitle').text.strip()

        # 내용 추출
        content = soup.find('div', id='powerbbsContent').text.strip()

        # 댓글 크롤링 (Selenium 사용)
        comments = crawl_comments_with_selenium(url)

        return {
            "title": title,
            "content": content,
            "comments": comments,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error on post_id {post_id}: {e}")

    return None

# 체크포인트 저장 함수
def save_checkpoint(data, start_id, batch_size, output_dir):
    filename = f"{output_dir}/maple_inven_{start_id}_{start_id + batch_size - 1}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 실패한 ID 저장
def save_failed_ids(failed_ids, output_dir):
    filename = f"{output_dir}/failed_ids.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(failed_ids, f, ensure_ascii=False, indent=4)

# 크롤링 실행 함수
def crawl_posts(start_id, end_id, batch_size, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성
    current_id = start_id
    data = []
    failed_ids = []

    # 이전에 실패한 ID 로드
    failed_ids_file = os.path.join(output_dir, "failed_ids.json")
    if os.path.exists(failed_ids_file):
        with open(failed_ids_file, "r", encoding="utf-8") as f:
            failed_ids = json.load(f)

    with ThreadPoolExecutor(max_workers=3) as executor:  # 최대 3개의 스레드 사용
        while current_id >= end_id:  # 최신 글부터 마지막 글까지 크롤링
            futures = {executor.submit(crawl_post, post_id): post_id for post_id in range(current_id, current_id - batch_size, -1) if post_id >= end_id}
            for future in tqdm(futures):
                post_data = future.result()
                if post_data:
                    data.append(post_data)
                else:
                    failed_ids.append(futures[future])  # 실패한 ID 기록

            # 배치 단위로 저장
            save_checkpoint(data, current_id, batch_size, output_dir)
            save_failed_ids(failed_ids, output_dir)  # 실패한 ID 저장
            current_id -= batch_size  # 다음 배치로 이동
            data = []  # 저장 후 데이터 초기화

    print("크롤링 완료!")

# 실행
crawl_posts(
    start_id=334410,  # 최신 게시글 번호
    end_id=334250,  # 마지막 게시글 번호
    batch_size=50,  # 20개 단위로 저장
    output_dir="maple_inven_data"  # 출력 디렉토리
)

  0%|          | 0/50 [00:00<?, ?it/s]

In [2]:
import os
import json
import requests
from bs4 import BeautifulSoup
import time

# 기본 설정
base_url = "https://arca.live"
category_url = f"{base_url}/b/maplestory?category=질문"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}
save_folder = "posts"
os.makedirs(save_folder, exist_ok=True)

def get_post_ids(page=1):
    """카테고리 페이지에서 게시글 번호 및 링크 추출"""
    page_url = f"{category_url}&p={page}"
    response = requests.get(page_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to fetch category page: {page_url}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    post_links = soup.find_all("a", class_="vrow column")
    print(f"Found {len(post_links)} posts on page {page}.")
    
    post_ids = []
    
    for link in post_links:
        post_id_tag = link.find("span", class_="vcol col-id").find("span")
        if post_id_tag:
            post_id = post_id_tag.text.strip()
            post_title = link.find("span", class_="title").text.strip()
            post_url = base_url + link["href"]
            print(f"Found post ID: {post_id}, Title: {post_title}")
            post_ids.append((post_id, post_url))

    return post_ids

def save_post_data(post_id, post_data):
    """게시글 데이터를 파일로 저장"""
    save_path = os.path.join(save_folder, f"post_{post_id}.json")
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(post_data, f, ensure_ascii=False, indent=4)
    print(f"Saved post {post_id} to {save_path}")

def crawl_post(post_url):
    """게시글 URL로부터 데이터를 크롤링"""
    response = requests.get(post_url, headers=headers)
    
    print(f"Accessing post: {post_url} (Status code: {response.status_code})")
    
    if response.status_code != 200:
        print(f"Failed to access post: {post_url}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    
    title = soup.find("div", class_="title")
    content = soup.find("div", class_="fr-view article-content")
    
    if not title or not content:
        print(f"Post at {post_url} is missing title or content.")
        return None

    comments = [
        comment.find("div", class_="text").text.strip()
        for comment in soup.find_all("div", class_="comment-item")
        if comment.find("div", class_="text")
    ]

    return {
        "title": title.text.strip(),
        "content": content.text.strip(),
        "comments": comments
    }
def main():
    for page in range(51, 91):  # 1페이지부터 50페이지까지
        post_ids = get_post_ids(page)
        if not post_ids:
            print(f"No more posts found on page {page}. Stopping.")
            break
        
        page_data = []
        for post_id, post_url in post_ids:
            print(f"Crawling post {post_id} from {post_url}")
            post_data = crawl_post(post_url)
            if post_data:
                page_data.append(post_data)
            time.sleep(8)  # 요청 간 딜레이
        
        # 페이지의 모든 게시글을 하나의 JSON 파일로 저장
        if page_data:
            save_page_data(page, page_data)
        
        time.sleep(8)  # 페이지 요청 간 딜레이

    print("Crawling completed.")


def save_page_data(page, page_data):
    save_path = os.path.join(save_folder, f"page_{page}.json")
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(page_data, f, ensure_ascii=False, indent=4)
    print(f"Saved page {page} data to {save_path}")

if __name__ == "__main__":
    main()

Found 45 posts on page 51.
Found post ID: 69539, Title: 할겜찾다 우연히 메이플이 눈에들어왔는데
Found post ID: 69538, Title: 웰컴 메이플 패키지
Found post ID: 69537, Title: 급함) 전사 18성 2초뚝 질문
Found post ID: 69536, Title: 카룻상의 가격볼때 어떤거 사는게 적당해보임?
Found post ID: 69535, Title: 환산 7만 단계에서 에디 공3퍼 차이 큼?
Found post ID: 69534, Title: 소마템 전부 파는게 정답일까요?
Found post ID: 69533, Title: 크뎀 장갑 아케인 vs 에테
Found post ID: 69532, Title: 숄더는 직작보다 완작된걸 사는게 맞나
Found post ID: 69531, Title: 아즈모스 해서 주화 어케 얻는거임?
Found post ID: 69530, Title: 아케인 레전 깡통 사도 ㄱㅊ?
Found post ID: 69529, Title: 템 바꿀거 훈수받고 싶은데 괜찮을까요?
Found post ID: 69528, Title: 흙에테는 무슨무슨 부위하는게 좋음?
Found post ID: 69527, Title: 보조 싸게 레전두줄 쓰다 넘어가는건 어떰
Found post ID: 69526, Title: 요즘 썬콜 상태 어떰?
Found post ID: 69525, Title: 오로란데 이거 67억 어떰?
Found post ID: 69524, Title: 이거 11억에 살만함?
Found post ID: 69523, Title: 제로 궁금한데 독수리 타자도 할 수 있을까?
Found post ID: 69522, Title: 카루타 바지~
Found post ID: 69521, Title: 스카니아 해적 카루타 dex120급 장비 얼마정도 하나요?
Found post ID: 69520, Title: 미트라는 언제 사나요?
Found post ID: 

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
import time

# 기본 설정
base_url = "https://arca.live"
category_url = f"{base_url}/b/maplestory?category=정보"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}
save_folder = "posts_1"
os.makedirs(save_folder, exist_ok=True)

def get_post_ids(page=1):
    """카테고리 페이지에서 게시글 번호 및 링크 추출"""
    page_url = f"{category_url}&p={page}"
    response = requests.get(page_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to fetch category page: {page_url}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    post_links = soup.find_all("a", class_="vrow column")
    print(f"Found {len(post_links)} posts on page {page}.")
    
    post_ids = []
    
    for link in post_links:
        post_id_tag = link.find("span", class_="vcol col-id").find("span")
        if post_id_tag:
            post_id = post_id_tag.text.strip()
            post_title = link.find("span", class_="title").text.strip()
            post_url = base_url + link["href"]
            print(f"Found post ID: {post_id}, Title: {post_title}")
            post_ids.append((post_id, post_url))

    return post_ids

def save_post_data(post_id, post_data):
    """게시글 데이터를 파일로 저장"""
    save_path = os.path.join(save_folder, f"post_{post_id}.json")
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(post_data, f, ensure_ascii=False, indent=4)
    print(f"Saved post {post_id} to {save_path}")

def crawl_post(post_url):
    """게시글 URL로부터 데이터를 크롤링"""
    response = requests.get(post_url, headers=headers)
    
    print(f"Accessing post: {post_url} (Status code: {response.status_code})")
    
    if response.status_code != 200:
        print(f"Failed to access post: {post_url}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    
    title = soup.find("div", class_="title")
    content = soup.find("div", class_="fr-view article-content")
    
    if not title or not content:
        print(f"Post at {post_url} is missing title or content.")
        return None

    comments = [
        comment.find("div", class_="text").text.strip()
        for comment in soup.find_all("div", class_="comment-item")
        if comment.find("div", class_="text")
    ]

    return {
        "title": title.text.strip(),
        "content": content.text.strip(),
        "comments": comments
    }
def main():
    for page in range(16, 30):  # 1페이지부터 50페이지까지
        post_ids = get_post_ids(page)
        if not post_ids:
            print(f"No more posts found on page {page}. Stopping.")
            break
        
        page_data = []
        for post_id, post_url in post_ids:
            print(f"Crawling post {post_id} from {post_url}")
            post_data = crawl_post(post_url)
            if post_data:
                page_data.append(post_data)
            time.sleep(8)  # 요청 간 딜레이
        
        # 페이지의 모든 게시글을 하나의 JSON 파일로 저장
        if page_data:
            save_page_data(page, page_data)
        
        time.sleep(8)  # 페이지 요청 간 딜레이

    print("Crawling completed.")


def save_page_data(page, page_data):
    save_path = os.path.join(save_folder, f"page_{page}.json")
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(page_data, f, ensure_ascii=False, indent=4)
    print(f"Saved page {page} data to {save_path}")

if __name__ == "__main__":
    main()

Found 45 posts on page 16.
Found post ID: 2095, Title: 퀴즈 13번 정답
Found post ID: 2094, Title: 니네 이거는 받고 게임하니
Found post ID: 2093, Title: 퀴즈 14번 정답
Found post ID: 2092, Title: 현재 밝혀진 퀴즈 정답
Found post ID: 2091, Title: 퀴즈 3번 정답
Found post ID: 2090, Title: 퀴즈 5번 정답
Found post ID: 2089, Title: 운영자 퀴즈쇼 2번정답
Found post ID: 2088, Title: 운영자 npc퀴즈 4번 정답
Found post ID: 2087, Title: 메이플 운영자 퀴즈 생김
Found post ID: 2086, Title: 스우 격파 문구 바뀌었네
Found post ID: 2085, Title: 이벤트 맵 여기 앉아짐!
Found post ID: 2084, Title: 메이플커마 아이템 업데이트
Found post ID: 2083, Title: 창조의 아이온과 파괴의 얄다바오트는
Found post ID: 2082, Title: (스?포)이번이벤트 히든업적
Found post ID: 2081, Title: 축구 120승이 폐기물급 이벤트인 이유
Found post ID: 2080, Title: 테라버닝으로 경험치 이벤트 드가는건 155렙이면 좋을뜻
Found post ID: 2079, Title: 싸전귀 필요 승리 횟수
Found post ID: 2078, Title: 미라클때 에디 직작할 사람 미세팁
Found post ID: 2077, Title: 저거 히든퀘 피로회복제는
Found post ID: 2076, Title: 이벤트맵 히든퀘 공략
Found post ID: 2075, Title: 뭔 히든미션 있나보네
Found post ID: 2074, Title: 이벤트맵 히든퀘
Found post ID: 2073, Title: 스우 이벤 보상
